# BioMQM Pipeline - Qwen2.5-3B-Instruct

**Pipeline:**
1. QG - Question Generation from source (English)
2. QA - Question Answering on source + backtranslation
3. Evaluation - String comparison + SBERT by severity
4. Decision - Accept/Reject based on severity and mismatches

**Languages:** de, es, fr, ru, zh-CN

**Compatible with:** Google Colab & Kaggle

## 0. Environment Setup

In [ ]:
import os
import sys
import subprocess

IN_COLAB = 'google.colab' in sys.modules
IN_KAGGLE = 'KAGGLE_KERNEL_RUN_TYPE' in os.environ

print(f'Running on: {"Colab" if IN_COLAB else "Kaggle" if IN_KAGGLE else "Local"}')

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    CACHE_DIR = '/content/drive/MyDrive/AskQE_Models_Cache'
elif IN_KAGGLE:
    CACHE_DIR = '/kaggle/working/cache'
else:
    CACHE_DIR = os.path.expanduser('~/.cache/huggingface')

os.makedirs(CACHE_DIR, exist_ok=True)
os.environ['HF_HOME'] = CACHE_DIR
os.environ['TRANSFORMERS_CACHE'] = os.path.join(CACHE_DIR, 'transformers')
print(f'Cache directory: {CACHE_DIR}')

In [ ]:
subprocess.run([sys.executable, '-m', 'pip', 'install', '-q', 
                'transformers', 'torch', 'accelerate', 'nltk', 
                'sentence-transformers', 'sacrebleu', 'textstat'], check=True)

REPO_URL = 'https://github.com/FedeBaldi-28/askqe-weighted-extension.git'

if IN_COLAB:
    PROJECT_ROOT = '/content/askqe'
elif IN_KAGGLE:
    PROJECT_ROOT = '/kaggle/working/askqe'
else:
    PROJECT_ROOT = os.getcwd()

if not os.path.exists(PROJECT_ROOT):
    subprocess.run(['git', 'clone', REPO_URL, PROJECT_ROOT], check=True)
    print(f'Repository cloned to: {PROJECT_ROOT}')
else:
    print(f'Repository already exists at: {PROJECT_ROOT}')

RESULTS_DIR = os.path.join(PROJECT_ROOT, 'results Qwen3B baseline')
os.makedirs(RESULTS_DIR, exist_ok=True)

---
## 1. Question Generation (QG)

In [ ]:
os.chdir(os.path.join(PROJECT_ROOT, 'QG', 'code'))
subprocess.run([sys.executable, '-u', 'qwen-3b-biomqm.py', '--prompt', 'vanilla'], check=True)
print('\n✓ QG completed!')

---
## 2. Question Answering (QA)

In [ ]:
os.chdir(os.path.join(PROJECT_ROOT, 'QA', 'code'))
subprocess.run([sys.executable, '-u', 'qwen-3b-biomqm.py', '--run_all'], check=True)
print('\n✓ QA completed!')

---
## 3. Evaluation

### 3.1 String Comparison (F1, EM, chrF, BLEU)

In [ ]:
os.chdir(os.path.join(PROJECT_ROOT, 'evaluation', 'string-comparison'))
subprocess.run([sys.executable, 'string_comparison_biomqm.py'], check=True)
print('\n✓ String comparison completed!')

### 3.2 SBERT (Cosine Similarity)

In [ ]:
os.chdir(os.path.join(PROJECT_ROOT, 'evaluation', 'sbert'))
subprocess.run([sys.executable, 'sbert_biomqm.py'], check=True)
print('\n✓ SBERT evaluation completed!')

---
## 4. Decision Rule

- Accept: Neutral, Minor severity
- Major/Critical: accept if mismatches <= threshold

In [ ]:
os.chdir(os.path.join(PROJECT_ROOT, 'biomqm'))

for n in range(1, 6):
    print(f'\n=== Threshold: {n} ===')
    subprocess.run([sys.executable, 'decision_biomqm.py', '--run_all', '--max_mismatches', str(n)], check=True)

print('\n✓ Decision rules applied!')

---
## Pipeline Complete!